## Import packages

In [72]:
import selenium
from selenium import webdriver

import shutil, time, os
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

from importlib import reload

import src
reload(src)

<module 'src' from 'C:\\Users\\Matth\\git\\DataAnalysisWorkbooks\\warcraftLogs\\src.py'>

## Configurations

In [2]:
path_to_extension = r'C:\Users\Matth\Desktop\3.12_0'
character_name = "Mercychan"
character_server = "Benediction"
character_region = "US"

clear_char_data = False

## Navigate to character page

In [3]:
char_url, browser = src.load_environment(path_to_extension, character_name, character_server, character_region)

In [123]:
bosses = src.get_phase3_bosses(browser)

In [39]:
# Feel free to omit bosses that you are not interested in
bosses = ["Anetheron"]#, "Anetheron", "The Illidari Council"]

In [62]:
boss_link_dict = {"High Warlord Najentus" : "#boss=601", "Supremus" : "#boss=602", "Shade of Akama" : "#boss=603", 
                  "Teron Gorefiend" : "#boss=604", "Gurtogg Bloodboil" : "#boss=605", "Reliquary of Souls" : "#boss=606", 
                  "Mother Shahraz" : "#boss=607", "The Illidari Council" : "#boss=608", "Illidan Stormrage" : "#boss=609", 
                  "Rage Winterchill" : "#boss=618", "Anetheron" : "#boss=619", "Kaz'rogal" : "#boss=620", 
                  "Azgalor" : "#boss=621", "Archimonde" : "#boss=622"}

In [17]:
char_info = src.get_character_info(char_url)
char_info

'Mercychan (Benediction US)'

## Main code loop

In [74]:
def main():
    if clear_char_data:
        src.clear_character_data()
    
    for i, boss in enumerate(bosses):
        
        player_df = pd.DataFrame(pd.np.empty((0, 16)))
        player_df.columns = ["Name", "Server", "Date", "Kill time", "Rank", "nHealers", "Spriest?", "Innervate?", "LB_uptime", "HPS", "% LB (tick) HPS", "% LB (bloom) HPS", "% Rejuv HPS", "% Regrowth HPS", "% Swiftmend HPS", "Rotations"]

        browser.get(f"https://classic.warcraftlogs.com/character/{character_region.lower()}/{character_server.lower()}/{character_name.lower()}{boss_link_dict[boss]}")
        time.sleep(1)

        # Pick out the top parse of this boss
        search = browser.find_elements_by_class_name("character-table-link")
        boss_page_url = browser.current_url
        
        print(f"Boss: {boss}")
        
        #for i in range(len(search)):
        for i in range(1):
                        
            try:
                rank, date, boss_rank, boss_HPS, boss_killtime = src.get_boss_data(browser, i)

            except IndexError: continue
                
            search = browser.find_elements_by_class_name("character-table-link")
                
            if src.check_if_parse_already_recorded(i, browser, search, boss, character_name, character_server, character_region): 
                print("Parse already recorded. Skipping")
                browser.get(boss_page_url)
                continue
            
            rank.click()
            time.sleep(2)
            
            temp_url = browser.current_url
            
            player_HPS = src.get_player_HPS(browser, character_name)
            
            boss_tanks = src.get_tanks(browser)
            nHealers = src.get_nHealers(browser)
            
            print(f"Tanks: {boss_tanks}")

            browser.get(temp_url)
            time.sleep(0.5)

            src.scroll_click(browser.find_element_by_link_text(character_name))
            time.sleep(0.5)
            
            
            LBtick_HPS, LBbloom_HPS, rejuv_HPS, regrowth_HPS, swiftmend_HPS, LB_uptime = src.get_spell_info(browser, player_HPS)
            time.sleep(2)
                
            spriest = src.check_spriest(browser)
            innervate = src.check_innervate(browser)

            # Download the cast-sequence and damage-taken CSVs.
            src.download_csv(browser, temp_url, "filter-casts-tab", "cast_sequence/casts.csv")
            src.download_csv(browser, temp_url, "filter-damage-taken-tab", "damage_taken/dmg_taken.csv")

            #nHealers = src.get_nHealers(browser)
            browser.get(temp_url)
            time.sleep(1)

            # For most single tank bosses, the main tank is the current boss' melee target
            if boss in ["Rage Winterchill", "Kaz'rogal", "Archimonde", "High Warlord Naj'entus", "Supremus", 
                        "Teron Gorefiend", "Gurtogg Bloodboil", "Mother Shahraz"]:

                # Clean the csvs
                df = src.clean_dmg_taken_csv(boss)
                df2 = src.clean_cast_sequence_csv()

                # Fix regrowth timer
                df2 = src.fix_regrowth_cast_time(df2)

                # Add boss target
                df = src.add_boss_target(boss, df, df2)
                df = src.fix_boss_target(df)

                # Get the rotations
                try: rotations = src.calculate_rotations(df, boss, boss_tanks)
                except ZeroDivisionError:
                    print("Click element error. Skipping this parse.")
                    browser.get(boss_page_url)
                    time.sleep(0.5)
                    continue

            elif boss in ["Anetheron", "Azgalor", "Reliquary of Souls",
                         "Shade of Akama", "The Illidari Council", "Illidan Stormrage"]:

                # Clean the csv
                df = src.clean_cast_sequence_csv()
                df = src.fix_regrowth_cast_time(df)

                # Get the rotations
                try: rotations = src.calculate_rotations(df, boss, boss_tanks)
                except ZeroDivisionError:
                    print("Click element error. Skipping this parse.")
                    browser.get(boss_page_url)
                    time.sleep(0.5)
                    continue

            print(f'Rotations: {rotations}')    

            # Save data to csv
            to_append = [character_name, character_server + " " + character_region, date, boss_killtime, boss_rank, str(nHealers), spriest, innervate, LB_uptime, boss_HPS, LBtick_HPS, LBbloom_HPS, rejuv_HPS, regrowth_HPS, swiftmend_HPS, rotations]
            series = pd.Series(to_append, index = player_df.columns)
            player_df = player_df.append(series, ignore_index=True)

            #Return to boss page
            browser.get(boss_page_url)
            time.sleep(0.5)
            print('------')
                
        # Return to character page
        browser.get(char_url)
        time.sleep(0.5)

        player_df.to_csv(f"character_data/{boss}_{character_name}.csv", index=None)
        
    src.combine_character_data(character_name, character_server, character_region, date, boss_rank, boss)
    src.add_row_to_xlsx(boss, character_name)
    src.sort_excel(boss)
    print(f"{boss} scraping complete.")
    print("---------------")
    
    os.remove(f"character_data/{boss}_{character_name}.csv")
    

if __name__ == "__main__":
    main()

Boss: Anetheron
Tanks: ['Maxload', 'Hilfedanke', 'Formulatank']
Rotations: [['Other', 0.333], ['1LB 2RG', 0.111]]
------
Anetheron scraping complete.
---------------
